<a href="https://colab.research.google.com/github/massinoLight/HandwrittenSignatureForgeryDetectionUsingCNN/blob/master/Handwritten_Signature_Forgery_Detection_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import une partie de la dataset du fichier zip qui est sur le github
import pathlib
import os
import cv2
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import datetime
from tensorflow import keras
from tensorflow.keras.models import Model
import tensorflow as tf
#Import dataset
import pathlib
import os
data_dir = tf.keras.utils.get_file(
    "datasets.zip",
    "https://github.com/massinoLight/HandwrittenSignatureForgeryDetectionUsingCNN/raw/master/datasets.zip",
    extract=False)

import zipfile
with zipfile.ZipFile(data_dir, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets')

data_dir = pathlib.Path('/content/datasets/datasets')
print(data_dir)
print(os.path.abspath(data_dir))

/content/datasets/datasets
/content/datasets/datasets


In [ ]:
image_count = len(list(data_dir.glob('*/*')))
print(image_count)

20


In [ ]:
batch_size = 3
img_height = 200
img_width = 200

train_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  )

val_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = val_data.class_names
print(class_names)

Found 18 files belonging to 2 classes.
Using 15 files for training.
Found 18 files belonging to 2 classes.
Using 3 files for validation.
['fake', 'true']
